In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install --quiet lightning
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 14.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import string
from sklearn.model_selection import train_test_split
import transformers
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

## with text only

In [7]:
# Define the parameters to log with MLflow
params = {
    "train_path": "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/kaggle_preprocessed.csv",
    "test_path": "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv",
    "test_path_dep": "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/test_data_preprocessed.csv",
    "model_name": 'distilbert-base-uncased',
    "num_labels": 2,
    "batch_size": 6,
    "lr": 5e-5,
    "epochs": 3
}

In [8]:
#Loading preprocessed Train and Test Data
train_data_processed = pd.read_csv(params["train_path"])
test_data_processed = pd.read_csv(params["test_path"])
test_data_dep_processed = pd.read_csv(params["test_path_dep"])

In [18]:
display(train_data_processed)

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
0,carfree cities become subject increasing inter...,1,7.063397,0.340870,6.083478,0.166957
1,car free cities carfree cities concept gaining...,1,7.512908,0.309942,6.298246,0.191033
2,sustainable urban future carfree cities emergi...,1,8.000000,0.302734,6.445312,0.197266
3,pioneering sustainable urban living era marked...,1,11.079196,0.333333,6.175337,0.190751
4,path sustainable urban living age rapid urbani...,1,11.405468,0.315068,6.207436,0.195695
...,...,...,...,...,...,...
27335,todays society everevolving role technology gr...,1,15.924749,0.289773,6.196023,0.125000
27336,everevolving world technology changed way indi...,1,13.415526,0.372881,5.658192,0.118644
27337,fostering rescue dog gaining unconditional lov...,1,17.275119,0.398827,5.149560,0.099707
27338,importance solid foundation education paramoun...,1,22.329353,0.354610,5.776596,0.109929


In [19]:
train_data_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27340 entries, 0 to 27339
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                27339 non-null  object 
 1   label               27340 non-null  int64  
 2   sentence_variation  27340 non-null  float64
 3   %stop_word_total    27340 non-null  float64
 4   mean_word_length    27340 non-null  float64
 5   %adjectives_total   27340 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.3+ MB


In [20]:
test_data_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                40 non-null     object 
 1   label               40 non-null     int64  
 2   sentence_variation  40 non-null     float64
 3   %stop_word_total    40 non-null     float64
 4   mean_word_length    40 non-null     float64
 5   %adjectives_total   40 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.0+ KB


In [21]:
test_data_dep_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                50 non-null     object 
 1   label               50 non-null     int64  
 2   sentence_variation  50 non-null     float64
 3   %stop_word_total    50 non-null     float64
 4   mean_word_length    50 non-null     float64
 5   %adjectives_total   50 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.5+ KB


In [22]:
# Split the dataset
df_train, df_validation = train_test_split(train_data_processed, test_size=0.3, random_state=42)
df_test = test_data_processed
df_test_dep = test_data_dep_processed

df_train.dropna(inplace=True)
df_validation.dropna(inplace=True)
df_test.dropna(inplace=True)
df_test_dep.dropna(inplace=True)

In [10]:
# Tokenization and data preparation
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [23]:
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        record = self.dataframe.iloc[idx]
        text = record['text']
        label = record['label']

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [24]:
# Create datasets and dataloaders
train_dataset = TextDataset(df_train, tokenizer)
val_dataset = TextDataset(df_validation, tokenizer)
test_dataset = TextDataset(df_test, tokenizer)
test_dep_dataset = TextDataset(df_test_dep, tokenizer)

train_loader = DataLoader(dataset=train_dataset, batch_size=params["batch_size"], shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=params["batch_size"])
test_loader = DataLoader(dataset=test_dataset, batch_size=1)
test_loader_dep = DataLoader(dataset=test_dep_dataset, batch_size=1)

In [25]:
class LightningModule(pl.LightningModule):
    def __init__(self, model_name, num_labels, learning_rate=5e-5):
        super().__init__()
        self.save_hyperparameters()
        self.model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        logits = self(input_ids, attention_mask)
        loss = F.cross_entropy(logits, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        logits = self(input_ids, attention_mask)
        loss = F.cross_entropy(logits, labels)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

In [26]:
lightning_module = LightningModule(params["model_name"], params["num_labels"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lightning_module.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LightningModule(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            

In [27]:
callbacks = [
    ModelCheckpoint(
        save_top_k=1,
        mode="min",
        monitor="val_loss"
    )  # save top 1 model
]

# Trainer configuration
trainer = pl.Trainer(
    max_epochs=params["epochs"],
    callbacks=callbacks,
    devices=1,
    logger=False
)

trainer.fit(lightning_module, train_loader, val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                                | Params
--------------------------------------------------------------
0 | model | DistilBertForSequenceClassification | 67.0 M
--------------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
267.820   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [28]:
# Save the trained model after training
torch.save(lightning_module, '/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/models/DistilBert/DistilBert_kaggle_nometa.pth')